In [ ]:
# SG2 - WS
# Topic: eye-tracking data preprocess & abstraction
# Done:  (1) Eye-tracking data preprocess & abstraction
#        (2) Fuse left & right eye

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import itertools 

In [ ]:
# Path
destination_folder = "D:/vr-project/.../data/result" #result
path_clean = "D:/vr-project/.../data/clean/SG2" #clean
clean_file = os.walk(path_clean) 

In [ ]:
# Create blank csv
df1 = pd.DataFrame(columns = range(7))
df1.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
save_info_path = destination_folder + '/SG2_info_FTN_1s.csv'
df1.to_csv(save_info_path, index=False, header=True) #default: header=False
#
df2 = pd.DataFrame(columns = range(21)) # 5  + 5 * 3 + 1 = 21
df2.columns = [
               "user_id", "game_id", "round_id", "device_id","block_id",
               "Quatx_max","Quaty_max", "Quatw_max",
               "Quatx_min","Quaty_min", "Quatw_min",
               "Quatx_mean","Quaty_mean", "Quatw_mean",
               "Quatx_std","Quaty_std", "Quatw_std", 
               "Quatx_median","Quaty_median", "Quatw_median",
               "IPD"
               ]
save_feat_path = destination_folder + '/SG2_ts_feature_FTN_1s.csv'   
df2.to_csv(save_feat_path, index=False, header=True)

In [ ]:
# (1) Divide block and abstract the information

for path, dir_list, file_list in clean_file:
  for file_name in file_list:

    filepath = os.path.join(path, file_name)
    raw = pd.read_csv(filepath)
    raw_data = raw.loc[:,["Orientation.x", "Orientation.y", "Orientation.w"]] # 3 meaningful features

    # get FBN_g 
    # FBN_g : how many blocks this ts need to be divided into
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]
    g_id = int(id_list[1])

    # Build ts blocks
    TS = 1 # TS = 1 means around one second per block, larger TS, less blocks, block size is bigger
    LEN = raw.shape[0]
    ts_delta = (int(raw.at[LEN-1, "Timestamp"]) - int(raw.at[0, "Timestamp"])) / pow(10,9)
    sample_rate = int(LEN/ts_delta) #floor
    granu = int(TS * sample_rate)
    df_wr = pd.DataFrame()

    # Traverse Blocks in a ts
    for i in range(1, int(ts_delta/TS) + 1):  
      
      #set block
      raw_block = raw_data[(i-1)*granu : (i*granu-1)]
      
      #compute statistics
      fea_block = [] # 5 statistics
      fea_block += list(raw_block.max())
      fea_block += list(raw_block.min())
      fea_block += list(raw_block.mean())
      fea_block += list(raw_block.std())
      fea_block += list(raw_block.median())
      fea_block += [IPD]
      df_block_fea = pd.DataFrame(fea_block)
      df_block_fea = df_block_fea.T
      
      #combine blocks feature
      str2 = file_name.split(".")
      str3 = str2[0].split("_")
      id_list = [int(element) for element in str3]
      id_list.append(i)
      
      id_df = pd.DataFrame(id_list)
      id_df = id_df.T
      id_df.columns = ["user_id", "game_id", "round_id", "device_id", "block_id"]
      df_block_row = pd.concat([id_df, df_block_fea], axis = 1, join = 'outer')
      df_wr = df_wr.append(df_block_row)
      
    # Save One file info
    str2 = file_name.split(".")
    str3 = str2[0].split("_")
    id_list = [int(element) for element in str3]

    id_list.append(ts_delta) # sec info: how many seconds
    id_list.append(LEN) # how many samples
    id_list.append(i+1) # how many blocks
    id_df = pd.DataFrame(id_list)
    id_df = id_df.T
    id_df.columns = ["user_id", "game_id", "round_id", "device_id", "time_slot", "sample_size", "block_number"]
    id_df.to_csv(save_info_path, mode='a', index=False, header=False)
    #time_slot: how long for user playing the game: Unit-second
    #sample_size:how many ts data
    #block_number:timeseries is divided into how many blocks
    
    # Save file featues
    df_wr.to_csv(save_feat_path, mode='a', index=False, header=False)

In [ ]:
# (2) Fuse to CSV
feature_path =  destination_folder + '/SG2_ts_feature_FTN_1s.csv' 
raw = pd.read_csv(feature_path)
df_4 = raw[(raw.device_id == 4)].drop(columns =["device_id"]) #left eye
df_6 = raw[(raw.device_id == 6)].drop(columns =["device_id"]) #right-eye
df_merge = pd.merge(df_4,df_6,how = "outer", on = ["user_id", "game_id", "round_id", "block_id", "IPD"])

pd.set_option('display.max_columns', None)
print(df_merge.shape)

In [ ]:
#Post process: Interpolate
for index, row in df_merge.iterrows():
    row_df = row.to_frame()
    if row_df.isnull().values.any():
        #find last block info
        row_last_find = df_merge[ (df_merge["user_id"] == row["user_id"]) &  (df_merge["game_id"] == row["game_id"]) & (df_merge["round_id"] == row["round_id"]) & (df_merge["block_id"] == row["block_id"] - 1)].copy()
        index_find = df_merge[ (df_merge["user_id"] == row["user_id"]) &  (df_merge["game_id"] == row["game_id"]) & (df_merge["round_id"] == row["round_id"]) & (df_merge["block_id"] == row["block_id"] - 1)].index.tolist()[0]
        df_merge.loc[index, "Quatx_max_x": "Quatw_median_y"] = row_last_find.loc[index_find, "Quatx_max_x": "Quatw_median_y"]
        print("*****")

In [ ]:
save_path = destination_folder + '/SG2_fuse_ts_feature_FTN_1s.csv' 
df_merge.to_csv(save_path, index=False, header=True)
print("SUCCESS SAVED File!")

In [ ]:
# Check result file size
feature_path =  destination_folder + '/SG2_fuse_ts_feature_FTN_1s.csv' 
raw = pd.read_csv(feature_path)
print(raw.shape)
print("READ SUCCESS")